## Importing packages

In [1]:
import sys
sys.path.append('/home/class_NI2021/ctxctl_contrib_2023')
import samna
import samna.dynapse1 as dyn1
from dynapse1constants import *
import dynapse1utils as ut
import netgen as n
import params
import time
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle
import os

## Looking for available devices

In [2]:
# Checking the list of unopened devices
devices = samna.device.get_unopened_devices()

if len(devices) == 0:
    raise Exception("no device detected!")

for i in range(len(devices)):
    print("["+str(i)+"]: ", devices[i], "serial_number", devices[i].serial_number)

[0]:  Bus 3 Device 10 Dynapse1DevKit serial_number 00000007
[1]:  Bus 3 Device 8 Dynapse1DevKit serial_number 00000031
[2]:  Bus 1 Device 23 Dynapse1DevKit serial_number 00000001
[3]:  Bus 1 Device 22 Dynapse1DevKit serial_number 00000000


In [3]:
# Select one device from the list
model,no_gui = ut.open_dynapse1(gui=False, sender_port=17654, receiver_port=17523, select_device=True) # returns Dynapse1Model

ERROR: Address already in use, please re-run open_device()!
[0]:  Bus 3 Device 10 Dynapse1DevKit serial_number 00000007
[1]:  Bus 3 Device 8 Dynapse1DevKit serial_number 00000031
[2]:  Bus 1 Device 23 Dynapse1DevKit serial_number 00000001
[3]:  Bus 1 Device 22 Dynapse1DevKit serial_number 00000000
Sender port:0 Dynapse1Wrapper created! libcaer init...
 tcp://0.0.0.0:17654
Receiver port: tcp://0.0.0.0:17523
Opened device name: Dynapse1DevKit
SamnaNode ID: 1
PythonNode ID: 2
Clearing chip 0... DONE.
Clearing chip 1... DONE.
Clearing chip 2... DONE.
Clearing chip 3... DONE.


In [12]:
import random

def AND_get_results(n_inputs, factor):
    # silent all neurons: enforces some biases to ensure neurons are NOT firing
    paramGroup = params.gen_clean_param_group()
    for chip in range(4):
        for core in range(4):
            model.update_parameter_group(paramGroup, chip, core)

    api  = model.get_dynapse1_api()

    # ---- Parameters ----
    n_delay = 1

    # ---- Spike generator ----
    spike_generator_ids = range(1, 4)  # SOLO 2 GENERADORES
    spike_generators = n.NeuronGroup(0, 0, spike_generator_ids, True)  # Avoid spike generator with ID 0

    # ---- DPI Neurons ----
    id_ini = 1
    id_end = id_ini + n_delay
    delay_pop = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    id_ini = id_end
    id_end += n_inputs
    not_pop = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    id_ini = id_end
    id_end += 1
    nor_pop = n.NeuronGroup(0, 1, range(id_ini, id_end), False)

    # ---- Synapses ----
    net_gen = n.NetworkGenerator()

    # OP delayed
    net_gen.add_connection(spike_generators.neurons[0], delay_pop.neurons[0], dyn1.Dynapse1SynType.AMPA)
    for i in range(1, n_delay):
        net_gen.add_connection(delay_pop.neurons[i - 1], delay_pop.neurons[i], dyn1.Dynapse1SynType.AMPA)

    # OP to NOT
    for i in range(n_inputs):
        net_gen.add_connection(spike_generators.neurons[0], not_pop.neurons[i], dyn1.Dynapse1SynType.AMPA)

    # OP to NOR
    net_gen.add_connection(delay_pop.neurons[-1], nor_pop.neurons[0], dyn1.Dynapse1SynType.AMPA)

    # Inputs to NOT
    for i in range(n_inputs):
        net_gen.add_connection(spike_generators.neurons[(i % 2) + 1], not_pop.neurons[i], dyn1.Dynapse1SynType.GABA_B)  # SOLO 2 GENERADORES

    # NOT to NOR
    for i in range(n_inputs):
        net_gen.add_connection(not_pop.neurons[i], nor_pop.neurons[0], dyn1.Dynapse1SynType.GABA_B)

    # make a dynapse1config using the network: that is convert the validated network to a Dynapse1 configuration
    new_config = net_gen.make_dynapse1_configuration()

    # apply the configuration
    model.apply_configuration(new_config)

    # Initialize custom parameters for a core and a chip:
    paramGroup = dyn1.Dynapse1ParameterGroup() 
    paramGroup.param_map["IF_THR_N"].coarse_value = 7
    paramGroup.param_map["IF_THR_N"].fine_value = 32
    paramGroup.param_map["IF_RFR_N"].coarse_value = 7  # Inverse
    paramGroup.param_map["IF_RFR_N"].fine_value = 255  # Inverse
    paramGroup.param_map["IF_TAU1_N"].coarse_value = 6  # Inverse
    paramGroup.param_map["IF_TAU1_N"].fine_value = 191  # Inverse
    paramGroup.param_map["IF_DC_P"].coarse_value = 0
    paramGroup.param_map["IF_DC_P"].fine_value = 0

    # Fast excitatory synapse (AMPA)
    paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 6
    paramGroup.param_map["NPDPIE_THR_F_P"].fine_value =  127
    paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
    paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value =  127
    paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
    paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 255

    # Fast inhibitory synapse (GABA_B)
    paramGroup.param_map["NPDPII_THR_S_P"].coarse_value = 6
    paramGroup.param_map["NPDPII_THR_S_P"].fine_value =  127
    paramGroup.param_map["NPDPII_TAU_S_P"].coarse_value = 4
    paramGroup.param_map["NPDPII_TAU_S_P"].fine_value = 127
    paramGroup.param_map["PS_WEIGHT_INH_S_N"].coarse_value = 7
    paramGroup.param_map["PS_WEIGHT_INH_S_N"].fine_value = 255

    # Apply the custom parameters to the core
    model.update_parameter_group(paramGroup, not_pop.chip_id, not_pop.core_id)

    fpga_spike_gen = model.get_fpga_spike_gen()
    
    # print(*np.sort(np.array(random.sample(op_times, 20))), sep=", ")
    op_times = [0,  1,  2,  4,  6,  7,  8,  9, 10, 12, 13, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 40, 41,
        42, 44, 45, 46, 48, 49]
    sg_times = [np.concatenate([np.array(op_times) / (1000 * factor) + 0.05, [0.400, 0.401]], axis=0)]

    in_times = []
    for i in range(2):  # SOLO 2 GENERADORES
        times = np.arange(20, 30)
        times = np.sort(list(set(np.append(times, random.sample(op_times, 20)))))
        in_times.append(times)
        sg_times.append(times / (1000 * factor) + 0.05)

    spike_times = np.concatenate(sg_times, axis=0)
    gen_neuron_ids = np.concatenate([[i+1] * len(sg_times[i]) for i in range(len(sg_times))], axis=0)  # Spike generator associated to the spike at that index
    spike_times, gen_neuron_ids = map(list, zip(*sorted(zip(spike_times, gen_neuron_ids), reverse=False)))

    post_neuron_chips = [0] * len(gen_neuron_ids)

    isi_base = 900
    repeat_mode = False
    ut.set_fpga_spike_gen(fpga_spike_gen, spike_times, gen_neuron_ids, post_neuron_chips, isi_base, repeat_mode)

    monitored_neurons = np.concatenate([delay_pop.tuple_neuron_ids, not_pop.tuple_neuron_ids, nor_pop.tuple_neuron_ids], axis=0)
    graph, filter_node, sink_node = ut.create_neuron_select_graph(model, monitored_neurons)

    api.reset_timestamp()
    graph.start()
    sink_node.get_events()
    fpga_spike_gen.start()

    if(spike_times[-1] - 0.2 < 0):
        time.sleep(spike_times[-1])
    else:
        time.sleep(spike_times[-1] - 0.2)  # Last spike time - 0.2

    fpga_spike_gen.stop()
    time.sleep(0.5)
    events = sink_node.get_events()
    graph.stop()

    evts_n = np.array([[evt.timestamp / 10**6 + spike_times[0], evt.neuron_id] for evt in events])

    save_array = [sg_times, evts_n]
    test_name = "inh_and_" + str(n_inputs) + "inputs_factor" + str(factor)
    folder_name = "results_AND"

    cwd = os.getcwd()
    if not os.path.exists(cwd + "/" + folder_name + "/"):
        os.mkdir(cwd + "/" + folder_name + "/")

    i = 1
    while os.path.exists(cwd + "/" + folder_name + "/" + test_name + "_" + str(i) + ".pickle"):
        i += 1

    filename = test_name + "_" + str(i)

    with open(folder_name + "/" + filename + '.pickle', 'wb') as handle:
        pickle.dump(save_array, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(folder_name + "/" + filename + '.pickle', "rb") as handle:
        print(pickle.load(handle))

    plt.rcParams['figure.dpi'] = 400
    plt.rcParams['figure.figsize'] = [9, 1.8 + 1.8 / 3 * n_inputs]

    plt.plot(sg_times[0], [1] * len(sg_times[0]), color='tab:blue', linestyle = 'None', marker='|', markersize=7)
    for i in range(n_inputs):  # SOLO 2 GENERADORES
        plt.plot(sg_times[(i % 2) + 1], [i+2] * len(sg_times[(i % 2) + 1]), color='tab:orange', linestyle = 'None', marker='|', markersize=7)
    plt.xlabel('Time (s)')
    plt.yticks(range(1, n_inputs + 3))
    plt.gca().set_yticklabels(["OP"] + ["Input " + str(i) for i in range(1, n_inputs + 1)] + ["Output"])
    plt.xlim([0.05 - 0.005 / factor, sg_times[0][-3] + 0.005 / factor])
    plt.ylim([0, n_inputs + 3])

    evts_nor = np.array([[evt.timestamp / 10**6 + spike_times[0], evt.neuron_id] for evt in events if evt.neuron_id in nor_pop.neuron_ids])
    if len(evts_nor) == 0:
        print("Output spikes were not detected")
    else:
        plt.plot(evts_nor[:,0], [n_inputs + 2] * len(evts_nor[:,0]), color='tab:green', linestyle = 'None', marker='|', markersize=7)

    plt.tight_layout()

    plt.savefig(folder_name + "/" + filename + '.png', transparent=False, facecolor='white', edgecolor='black')

    with open(folder_name + '/log.txt', 'a') as f:
        intersection = np.array(op_times) / (1000 * factor) + 0.05
        for i in range(2):  # SOLO 2 GENERADORES
            intersection = np.intersect1d(intersection, sg_times[(i % 2) + 1])

        if len(evts_nor) != 0:
            evts_nor_join = evts_nor[:,0]
            if len(evts_nor_join[evts_nor_join < 0.39]) == len(intersection[intersection < 0.39]):
                f.write("Number of inputs: " + str(n_inputs) + ", Factor: " + str(factor) + ", OK!\n")
            else:
                f.write("Number of inputs: " + str(n_inputs) + ", Factor: " + str(factor) + ", FAILED!\n")
        else:
            f.write("Number of inputs: " + str(n_inputs) + ", Factor: " + str(factor) + ", FAILED!\n")

    del op_times, sg_times, events
    plt.clf()

### EXPERIMENTS

In [13]:
reps = 3
n_input_list = [2, 3, 5, 10, 15, 20, 25, 30, 40, 50, 60]
n_factor_list = [0.5, 1, 2, 5, 10]

for i in range(reps):
    for j in n_input_list:
        for k in n_factor_list:
            AND_get_results(j, k)
            time.sleep(0.5)

New configuration applied to DYNAP-SE1!
VariableIsiMode already 1
RepeatMode already 0
[[array([0.05 , 0.052, 0.054, 0.058, 0.062, 0.064, 0.066, 0.068, 0.07 ,
       0.074, 0.076, 0.08 , 0.082, 0.084, 0.086, 0.088, 0.09 , 0.092,
       0.094, 0.096, 0.098, 0.1  , 0.102, 0.104, 0.106, 0.108, 0.11 ,
       0.112, 0.116, 0.122, 0.124, 0.126, 0.13 , 0.132, 0.134, 0.138,
       0.14 , 0.142, 0.146, 0.148, 0.4  , 0.401]), array([0.05 , 0.052, 0.062, 0.064, 0.07 , 0.076, 0.082, 0.086, 0.09 ,
       0.092, 0.094, 0.096, 0.098, 0.1  , 0.102, 0.104, 0.106, 0.108,
       0.11 , 0.112, 0.122, 0.124, 0.142, 0.146, 0.148]), array([0.05 , 0.052, 0.054, 0.058, 0.062, 0.066, 0.086, 0.088, 0.09 ,
       0.092, 0.094, 0.096, 0.098, 0.1  , 0.102, 0.104, 0.106, 0.108,
       0.124, 0.126, 0.13 , 0.132, 0.138, 0.14 , 0.142])], array([[0.050343, 1.      ],
       [0.050418, 4.      ],
       [0.052344, 1.      ],
       [0.052418, 4.      ],
       [0.054344, 1.      ],
       [0.054345, 2.      ],
       [0

/tmp/ipykernel_241021/723138932.py:173: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


New configuration applied to DYNAP-SE1!
VariableIsiMode already 1
RepeatMode already 0
[[array([0.05  , 0.0505, 0.051 , 0.052 , 0.053 , 0.0535, 0.054 , 0.0545,
       0.055 , 0.056 , 0.0565, 0.0575, 0.058 , 0.0585, 0.059 , 0.0595,
       0.06  , 0.0605, 0.061 , 0.0615, 0.062 , 0.0625, 0.063 , 0.0635,
       0.064 , 0.0645, 0.065 , 0.0655, 0.0665, 0.068 , 0.0685, 0.069 ,
       0.07  , 0.0705, 0.071 , 0.072 , 0.0725, 0.073 , 0.074 , 0.0745,
       0.4   , 0.401 ]), array([0.05  , 0.052 , 0.0535, 0.054 , 0.056 , 0.06  , 0.0605, 0.061 ,
       0.0615, 0.062 , 0.0625, 0.063 , 0.0635, 0.064 , 0.0645, 0.065 ,
       0.0655, 0.0665, 0.0705, 0.072 , 0.0725, 0.073 , 0.074 , 0.0745]), array([0.0505, 0.051 , 0.053 , 0.0535, 0.0545, 0.055 , 0.056 , 0.059 ,
       0.0595, 0.06  , 0.0605, 0.061 , 0.0615, 0.062 , 0.0625, 0.063 ,
       0.0635, 0.064 , 0.0645, 0.068 , 0.07  , 0.0705, 0.072 , 0.073 ,
       0.074 ])], array([[0.050331, 1.      ],
       [0.050335, 3.      ],
       [0.050831, 1.      ]

<Figure size 3600x1200 with 0 Axes>

## Close the device

In [8]:
# remeber to close the device
samna.device.close_device(model)